 # Analyse de la centralité des noeuds

On veut faire un programme qui étant donné un graphe retourne une analyse de la centralité des sommets du graphe en termes de degré, d'eccentricité, de centralité de proximité et de centralité d'intermédiarité.

Un script centralite.py qui accepte des arguments.
Notre programme prendra en entrée un fichier CSV contenant les listes d'adjacences définissant un graphe.
Notre programme devra afficher les n(=5 par defaut) sommets les plus centraux suivant les critères suivant:

      
    * les plus haut degrés
    * les plus petites excentricités
    * les plus grandes centralité de proximité
    * les plus grandes centralité d'intemédiarité
    
Notre programme devra également créer des fichiers: 
    
    * degres.csv
    * excentricites.csv
    * cproximites.csv 
    * intermediarites.csv
avec les statistiques correspondantes pour chacun des sommets, triés du plus central au moins central et par ordre lexicographique.


# Plan
     I- Création du script
     II- Algorithme de récupération des fichier csv
     III- Algorithmes de mesure de la centralité
         1) Degrés des chaque neouds du graphes
         2)
         3)
         4)
     IV- Création des fichier csv
     V-  La sortie du scripte
            

# Noté bien:
Tous les implémentations ci-dessous doivent être mise dans un même bloc de code pas des différentes bloque comme je le fais. J'ai repartie les implémentation en bloc pour explicité les différentes partie et travail abordés tout au long du travail. Après ça, téléchargé le fichier en extension **.py** et vous pouvez l’utiliser en l’appelant le scripte sur un terminal suivit d'un fichier csv qui définit le graphe qu'on veut étudier.

# I- Creation du script

In [ ]:
import argparse # pour pouvoir passer des arguments au script 
import csv
# Une description de ce que fait le script
parser = argparse.ArgumentParser(description='Affiche la chaîne de caractères' 
                                 +' passée en argument N fois'+'passée en argument N fois')

# Un argument obligatoire
#------- Le programme prend en entré un fichier csv (==filename) de type str == chaine , obligatoire
#------- contenant les listes d'adjacences définissant un graphe             ------

parser.add_argument('filename', metavar='c', type=str, nargs='?',
                    help='la chaine de caractères')

#-----cette ligne permet d'afficher les n noeuds (-Nn) passer en arguments, par defaut il affiche 5 noeuds (-N)
parser.add_argument('-N', dest='n', action='store', default=5, type=int,
                    help="nombres d'affichages de la chaîne (5 par defaut)")

# Lecture des arguments passés en paramètres
args = parser.parse_args()

# On affecte le fichier csv, qui sera passer en entré, par filename:
filename=args.filename

# II- Algorithme de récupération des fichier csv
      Implémenter une fonction qui prendra en entré un fichier csv définissant un graphe.
      Cette fonction doit faire en sorte que le graphe soit un graphe non-dirigé quelque
      la nature du graphe.

Pour le fiche csv donné en entré (obligatoire), on a la fonction qui prend les nœuds et les liste d'adjacence, et les met dans dictionnaire  dont les clé sont les nœuds concernés et les valeurs sont les liste adjacence des clé respectif.

In [ ]:
def recuperer_liste_adjacence(f):
    dictio=dict()
    with open(f, 'r') as file:
        reader = csv.reader(file)
        for ligne in reader:
            # premiere collonne qui contient les noeuds
            noeud = ligne[0]   
            # toute les collones apres la premiere qui sont les listes des adjacences
            list_adjacence = ligne[1:] 
            # stocker les ligne (les adjacent) comme valeur de la clé noeud
            dictio[noeud]=list_adjacence
            
    # On va s'assurer que les connexion entre les arret sont assurer deux fois, 
    # C'est à dire qu'on considere toute graphe comme non-dirigé
    les_clé=list(dictio.keys()) 
    for noeud in les_clé:
        for point in dictio[noeud]:
            if point not in dictio.keys():
                dictio[point]=list()
                dictio[point].append(noeud)
            if noeud not in dictio[point]:
                dictio[point].append(noeud) 
                
    return dictio

On applique la fonction avec le fichier csv affecté à filename.
Donc **liste_adj** est un dictionnaire portant les nœuds (clés) et listes des adjacences (valeurs). Rappelons que **filename** est le fichier csv mise entré dans le script.

In [ ]:
liste_adj=recuperer_liste_adjacence(filename)

# III- Algorithmes de mesure de la centralité
      1) Degres des chaque neouds du graphe

On veut formé un dictionnaire dont les valeurs sont les degres de chaque sommet et les cles sont les sommets respectifs.
La fonction suivante, calculate_degrees, prend comme arguments le dictionnaire des listes d'adjacent et retourne le 
degre de chaque nœuds sous forme d'un dictionnaire.

      le degré d'un nœud (clé du dictionnaire) est égale au nombre d'arets
      qui lui est lié donc c'est la longueur de la liste sa valeur dans le dictionnaire.

In [ ]:
def calculate_degrees(s):
    #-- le degré d'un noeud (clé du dictio) est egale au nombre d'arret-----
    #--- qui lui est lié donc c'est la longueur de la liste sa valeur dans la dictionnaire -----
    degrees = {noeud: len(list_adjacence) for noeud, list_adjacence in s.items()}

    return degrees


D=calculate_degrees(liste_adj)

La fonction qui donne les sommets les plus centraux selon le critère du plus haut degrés.

In [ ]:
def plus_haut_degrés(degre_dictio):
    sorted_keys = sorted(degre_dictio.keys(), key=lambda k: (-degre_dictio[k], k))
    degr = {}
    for key in sorted_keys:
        degr[key] = degre_dictio[key]
    return degr

plus_haut_degrés(calculate_degrees(liste_adj))

      2)  Excentricités de chaque sommets
L' **excentricités** d'un sommet est la plus petite distance entre le sommet et tout les autres sommets.
On commence par implement un algorithme qui calcule pour un sommet donné, elle calcule la distance entre ce sommet et tout les autre sommets:

In [ ]:
def Distance_BFS(graph, point_depart):
    
    # on a initialise le dictionnaire des distances a None cela veut dire que la distance est pas encore verifie
    distances = {noeud: None for noeud in graph} 
    
    # Nous considerons tous les sommets dans une liste d'attente dont le 1er est le sommet de depart
    # Chaque fois qu'une sommet est visité, il sort de la liste d'attente
     
    distances[point_depart] = 0            # on inisialise noeuds de depart a 0 car sa distance de lui meme c'est 0
    list_attente = [point_depart]          # on ajoute le point visité a la liste d'attente 
    while list_attente:                    # tanq que la liste d'attente n'est pas vide 
        element = list_attente.pop(0)      # alors defile le premier element de la liste 
        for voisin in graph[element]:      # on parcours la liste des adjacent du Noeuds defilé
            if distances[voisin] == None:  # si la distance est pas encore verifié  
                distances[voisin] = distances[element] + 1  # alors affecte 1 
                list_attente.append(voisin)  # et a la fin on ajoute a laliste d'attente le noeuds visité pour qu'on  
                                             # applique le meme algorithme sur les noeuds suivant

    return distances                        # finalement retourne la distance entre le noeuds de depart pris en
                                            # arguments a tous les autres Noeuds.

On va calcul les distance de chaque point du graphe à tous les autres du graphe en faisant appel à l'algo Distance_BFS précédent.
En gros cette fonction nous donne un dictionnaire dont les valeurs sont des dictionnaire portant comme clé 
Les autres nœuds et comme valeurs la distance avec la clé du dictionnaire d'origine.

In [ ]:
def calculate_all_distances(graph):
    distances = {}
    for noeud in graph.keys():
        
        # On parcourt les sommets, et chaque sommet donné par la boucle, 
        # on l'applique à l'algorithme Distance_BFS comme noeud de depart.
        distances[noeud] = Distance_BFS(graph,noeud )

    #---On returne un dictionnaire portant comme clé chaque noeud et comme 
    #---valeurs un dictionnaire portant les sommets et la distance ce sommet et le clé (sommet) respectif.
    return distances

bfs_distance=calculate_all_distances(liste_adj)

Jusqu'à la, on a calculé pour chaque sommet, sa distance à tous les autres sommets.
On calcule les excentricités des sommets: A partir du dictionnaire donné avant **bfs_distance**, on parcourt les valeurs et on cherche la valeur maximal car l'**excentricité** d'un sommet est la **distance maximal** entre le sommet et tout les autre sommets.

In [ ]:
def calcul_excentricités(graph):
    dictio=dict()
    for cle, val_distances in graph.items():
        l=list()
        for distance in val_distances.values():
            l.append(distance)
        dictio[cle]=max(l)
    return dictio

#----Jusqu'à là, calcul_excentricités prend le dictionnaire ayant comme clés les noeuds et comme valeurs 
#----les excentricités respectifs   ------

excentricités=calcul_excentricités(bfs_distance)

La fonction qui donne les sommets les plus centraux selon le critère du plus petit excentricité.

In [ ]:
def plus_petites_excentricités(excentricite_dictio):
    sorted_keys = sorted(excentricite_dictio.keys(), key=lambda k: (excentricite_dictio[k], k))
    excentrite = {}
    for key in sorted_keys:
        excentrite[key] = excentricite_dictio[key]
    return excentrite

plus_petites_excentricités(excentricités)

     3) Centralité de proximité
     
La **centralité de proximité** mesure à quel point un sommet X est connecté aux autres sommets. Plus un sommet à des voisins proches plus il est considéré comme un **point central** en terme de **centralité de proximité**.

Cette fonction s'appuie sur la fonction **calculate_all_distances()**, définie avant, pour calculer les **centralités de proximités**.

In [ ]:
def calculate_centralité_proximité(graph,n):
    dictio=dict()
    # on parcoure le dictionnaire qui contient les distantes de chaque noeuds aux autre Noeuds
    for cle, val_distances in graph.items(): 
        # creation d'une liste pour la remplir les distance, au fur et à mesure
        l=list()                             
        for distance in val_distances.values():  
            # on ajoute a la liste l les distances du noeuds parcouru en haut (cle) aux autres   
            l.append(distance)  
        # quand on ajoute toute les distances a liste l o essaye de retouner le maximum pour calculer la centralite proximité                
        dictio[cle] = ((n-1) / sum(l))  

     # et finalement on retourne dictionnaire dont les cle sont les noeuds et les valeurs sont la centralite proximité respectifs
    return dictio                        

centralité_proximité=calculate_centralité_proximité(bfs_distance,longeur_graphe(liste_adj))

La fonction qui donne les sommets les **plus centraux** selon le critère de la **plus grandes centralité de proximité**.

In [ ]:
def plus_grandes_centralité_proximité(central):
    sorted_keys = sorted(centralité_proximité.keys(), key=lambda k: (-centralité_proximité[k], k))
    proximité = {}
    for key in sorted_keys:
        proximité[key] = centralité_proximité[key]

    return proximité

cproximité=plus_grandes_centralité_proximité(centralité_proximité)

       4) Centralité d'intemédiarité
       
La **centralité d'intemédiarité** mesure la capacité d'un sommet X a être un point passager obligatoire entre les autres points. Plus un sommet est impliqué dans des nombreuse chemins plus il est considéré comme un point central en terme de **centralité d'intemédiarité**.

On a besoin de la fonction **Distance_BFS** avant mais rajoutée d'une troisième argument qui jouera un rôle important pour la calcule des centralité d'intemédiarité:

In [ ]:
from collections import defaultdict
 
def Distance_BFS2(graph, point_depart, argument):
    #--- Ne vous pose pas de question de cette fonction, c'est juste la mem avec la fonction Distance_BFS 
    #---- precedent sauf qu'on rajoute un 3eme argument argument qui sear utile pour la focntion qui suive -----
    distances = {noeud: None for noeud in graph}
    distances[point_depart] = 0
    list_attente = [point_depart]
    while list_attente:
        element = list_attente.pop(0)
        for voisin in graph[element]:
            if distances[voisin] == None:
                distances[voisin] = distances[element] + 1
                list_attente.append(voisin)

    return distances 

def Centralité_intermediaire(graph):
    # Initialise le dictionnaire des centralités intermediare avec la valeur par défaut 0
    Centralité = defaultdict(lambda: 0)
    # On boucle sur tous les nœuds du graphique
    for point_depart in graph.keys():
        # Initialiser le dictionnaire des distances avec les distances depuis le nœud de départ
        distances = Distance_BFS2(graph, point_depart, None)
        # On boucle sur tous les nœuds du graphe
        for cle in graph.keys():
            # On boucle sur tous les nœuds du graphe
            for noeud in graph.keys():
                # Si la distance du nœud de départ au nœud est égale à la distance du nœud de départ
                # au nœud de fin moins 1 et la distance du nœud au nœud final est égale à la distance 
                # du nœud de départ au nœud final moins la distance du nœud de départ au nœud 
                # incrémenter la centralité du nœud de 1
                if distances[noeud] == distances[cle] - 1 and 
                         distances[cle] - distances[noeud] == Distance_BFS2(graph, noeud, cle)[cle]:
                    Centralité[noeud] += 1
                    
    # On diviser, et mettre à jour les valeurs, les centralités par 2 pour tenir compte du double comptage
    for noeud in Centralité.keys():
        Centralité[noeud] /= 2
    
    return Centralité
 

intermediarité=Centralité_intermediaire(liste_adj)

La fonction qui donne les sommets les **plus centraux** selon la **plus grande centralité d'intemédiarité**:

In [ ]:
def plus_haut_intermediarités(interm):
    sorted_keys = sorted(interm.keys(), key=lambda k: (-interm[k], k))
    intemédiarité = {}
    for key in sorted_keys:
        intemédiarité[key] = interm[key]
    return intemédiarité

Intermedia=plus_haut_intermediarités(intermediarité)

# IV- Création des fichier csv
On va stocker les sommets dans un fichier csv, par trie du plus central au moins central et par ordre lexicographique à partir des différentes fonctions définies avant:

In [ ]:
def stocker_graph(filenames, liste_adjacence):
    with open(filenames, 'w', newline='') as file:
        writer = csv.writer(file)
        for noeud, list_adjacence in liste_adjacence.items():
            row = [noeud] + [list_adjacence]
            writer.writerow(row)


#----Pour le fichier degres.csv, on a:---
stocker_graph("degres.csv",plus_haut_degrés(calculate_degrees(liste_adj)))

#----Pour le fichier excentricites.csv, on a:---
stocker_graph("excentricites.csv",plus_petites_excentricités(excentricités))

#----Pour le fichier cproximites.csv, on a: ----
stocker_graph("cproximites.csv",cproximité)

#----Pour le fichier intermediarité.csv, on a:---
stocker_graph("intermediarité.csv",Intermedia)

 # V-  La sortie du scripte

Recuperons d'abord nos fichier CSV:

In [ ]:
def recuperer_liste_adjacence(filename):
    noeud=list()
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        for ligne in reader:
            noeud.append(ligne[0]) 
        return noeud
    

fichier_degre=recuperer_liste_adjacence("degres.csv")
fichier_excentricités=recuperer_liste_adjacence("excentricites.csv")
fichier_proximité=recuperer_liste_adjacence("cproximites.csv")
fichier_intermediaire=recuperer_liste_adjacence("intermediarité.csv")

Affichage des n(=5 par defaut) sommets les plus centraux, selon les différents critères définies dans les algorithmes précédents:

In [ ]:
def affichage(Noeuds, n):
    for j in Noeuds[:n]:
        print(j)


print("Les ",args.n," noeuds les plus centraux selons le critere du plus haut degres sont:")
affichage(fichier_degre, args.n)

print("Les",args.n," noeuds les plus centraux selons le critere du plus petites excentricités sont:")
affichage(fichier_excentricités, args.n)


print("Les ",args.n," noeuds les plus centraux selons le critere du plus grandes centralité de proximité sont:")
affichage(fichier_proximité, args.n)

print("Les ",args.n," noeuds les plus centraux selons le critere du plus grande centralité d'intemédiarité sont:")
affichage(fichier_intermediaire, args.n)